<a href="https://colab.research.google.com/github/XingxinHE/Chief_Curator_Griffin/blob/master/%5BFAIL_%26_ANALYSIS%5D_03_Griffin_Try_to_distinguish_the_type_of_art.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
import os, shutil
os.chdir("/content/gdrive/My Drive/Kaggle/Rijksmuseum")

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
ls

'01#Loading Info From XML'                        jpg/                  xml/
'02#Discover Interesting Data'                    project_curator.csv
'03#Griffin Try to distinguish the type of art'   type_df.csv


## Import the csv

In [ ]:
df = pd.read_csv('type_df.csv', index_col=0)

Reduce the size for test demo

In [ ]:
df_small = df.sample(n=100, random_state=13)
df_small

,type,tag,prent,tekening,boekillustratie,ornament,schilderij,historie,foto
5951,kan,0005955_BK-KOG-554,False,False,False,False,False,False,False
46636,prent,0046776_RP-P-1907-3144,True,False,False,False,False,False,False
41635,prent,0041768_RP-P-1904-358,True,False,False,False,False,False,False
47173,prent,0047315_RP-P-OB-48.475,True,False,False,False,False,False,False
62145,tekening,0062328_RP-T-FM-19,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...
45209,prent,0045345_RP-P-1964-209,True,False,False,False,False,False,False
96268,prent,0096578_RP-P-OB-9923,True,False,False,False,False,False,False
102080,"['prent', 'spotprent']",0102429_RP-P-OB-81.847A,True,False,False,False,False,False,False
36554,prent,0036659_RP-P-1905-4438,True,False,False,False,False,False,False


## Set Up

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from keras.layers import BatchNormalization
import tensorflow as tf

## ❌Wrong Method 1: Input X load image as Numpy Arrary

In [ ]:
#Because the small dataset is randomly sample which make the index is not sorted
df_small.reset_index(drop=True, inplace=True)

In [ ]:
#change to image directory
os.chdir('/content/gdrive/My Drive/Kaggle/Rijksmuseum/jpg')

为何下面这个不行？❌❌❌

1. rijks这个dataset十分巨大，13GB的图片，如果全部转化成np ndarray是行不通的（如下面这个做法）

2. 以下做法仅适合于小型的图片库

3. 这种做法相当于把所有数据一次性加载到神经网络，这样不仅是RAM， GPU也不行

In [ ]:
SIZE = 300
X_dataset = []  
for i in tqdm(range(df_small.shape[0])):
    img = image.load_img(df_small['tag'][i]+'.jpg', target_size=(SIZE,SIZE,3))
    img = image.img_to_array(img)
    img = img/255.
    X_dataset.append(img)
    if i > 100:
      break
    
X = np.array(X_dataset)

## ❌Wrong Method 2: Input X extract image to numpy at once

❌❌❌
这种错误的做法是上文做法的延申，该做法设想把所有图片都解码为numpy ndarray，然后一次性加载到神经网络里面，这是绝对不可以的。1000张图片就算设置成256*256，解码成ndarray之后也是1GB

In [ ]:
path_name = []
for i in range(df.shape[0]):
  path_name.append(str(df.iloc[i, 1]+".jpg"))


In [ ]:
len(path_name)

112039

In [ ]:
#chunk the images into 110 chunks
path_name = np.array_split(np.array(path_name), 110)

In [ ]:
path_name[13]

array(['0013263_RP-P-OB-1541.jpg', '0013264_RP-P-OB-1542.jpg',
       '0013265_RP-P-OB-1543.jpg', ..., '0014293_RP-P-H-233.jpg',
       '0014294_RP-P-H-234.jpg', '0014295_RP-P-OB-11.957.jpg'],
      dtype='<U39')

In [ ]:
len(path_name)

110

In [ ]:
#First loop to the chunk
for i in range(len(path_name)):
  image_ndarray = np.zeros((1,256,256,3))
  for path in path_name[i]:
    image = tf.keras.preprocessing.image.load_img(path, target_size=(256, 256))
    input_arr = keras.preprocessing.image.img_to_array(image)
    input_arr = input_arr/255.
    input_arr = input_arr.reshape((1,256,256,3))
    image_ndarray = np.vstack((image_ndarray, input_arr))
    print("Finish: \t"+str(path))
    del image
    del input_arr
  image_ndarray = image_ndarray[1:,:,:,:]
  np.save(('image_ndarray'+str(i)), image_ndarray)
  del image_ndarray

Streaming output truncated to the last 5000 lines.
Finish: 	0017404_RP-P-1936-820.jpg
Finish: 	0017405_RP-P-BI-4211.jpg
Finish: 	0017406_RP-P-H-R-62.jpg
Finish: 	0017407_RP-P-H-R-63.jpg
Finish: 	0017408_RP-P-BI-4212.jpg
Finish: 	0017409_RP-P-1887-A-11444.jpg
Finish: 	0017410_RP-P-BI-4213.jpg
Finish: 	0017411_RP-P-BI-4210A.jpg
Finish: 	0017412_RP-P-1887-A-11445.jpg
Finish: 	0017413_RP-P-1895-A-18694.jpg
Finish: 	0017414_RP-P-1878-A-2814.jpg
Finish: 	0017415_RP-P-BI-4202.jpg
Finish: 	0017416_RP-P-1898-A-20728.jpg
Finish: 	0017417_RP-P-1886-A-10697.jpg
Finish: 	0017418_RP-P-1895-A-18695.jpg
Finish: 	0017419_RP-P-BI-4203.jpg
Finish: 	0017420_RP-P-BI-4204.jpg
Finish: 	0017421_RP-P-BI-4205.jpg
Finish: 	0017422_RP-P-1895-A-18696.jpg
Finish: 	0017423_RP-P-BI-4206.jpg
Finish: 	0017424_RP-P-1878-A-2573.jpg
Finish: 	0017425_RP-P-H-H-1293.jpg
Finish: 	0017426_RP-P-1895-A-18697.jpg
Finish: 	0017427_RP-P-BI-4207.jpg
Finish: 	0017428_RP-P-BI-4818.jpg
Finish: 	0017429_RP-P-BI-4819.jpg
Finish: 	0017430

In [ ]:
image_ndarray = np.zeros((1,256,256,3))

In [ ]:
image_ndarray.shape